<a href="https://colab.research.google.com/github/mmsamiei/thesis-prototype/blob/master/phase2/squad_dataset_knowledgeselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2020-01-25 11:22:36--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  48.4MB/s    in 0.8s    

2020-01-25 11:22:37 (48.4 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [0]:
!pip install transformers

     |████████████████████████████████| 450kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 50.2MB/s 
     |████████████████████████████████| 870kB 60.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884629 sha256=a3458d341722a2356c166af6622300b68f2dcd906a65fbecfa92d36ffe4bf850
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [0]:
def text_to_idslist(text):
  text = str(text)
  text = "[CLS] " + text + " [SEP]"
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [0]:
import json

data = json.load(open('train-v2.0.json'))['data']

In [0]:
!pip install -U tqdm

     |████████████████████████████████| 61kB 2.0MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
from tqdm.notebook import tqdm
new_dataset = []


for topic in tqdm(data):
  paragraphs = topic['paragraphs']
  for paragraph in paragraphs:
    context = paragraph['context']
    sentences = context.split('.')
    for qa in paragraph['qas']:
      question = qa['question']
      question_tokenized = text_to_idslist(question)
      answers = qa['answers']
      if answers != []:
        answer = answers[0]['text']
        answer_tokenized = text_to_idslist(answer)
        for sentence in sentences:
          if answer not in sentence:
            history = question
            true_sentence = answer
            false_sentence = sentence
            false_sentence_tokenized = text_to_idslist(false_sentence)
            new_row = {}
            new_row['history'] = question_tokenized
            new_row['true_sentence'] = answer_tokenized
            new_row['false_sentenc'] = false_sentence_tokenized
            new_dataset.append(new_row)
      else:
        continue

In [0]:
len_dataset = len(new_dataset)

print(len_dataset)

486309


In [0]:
with open('squad_knowledgeselection_distilbert.json', 'w') as outfile:
  json.dump(new_dataset, outfile)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········


In [0]:
!cp /content/squad_knowledgeselection_distilbert.json /content/drive/My\ Drive/Thesis/phase-2

cp: cannot create regular file '/content/drive/My Drive/Thesis/phase-2': No such file or directory
